In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os 
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data")

In [ ]:
def generator(z, reuse=False):
    with tf.variable_scope('generator',reuse=reuse):
        with tf.variable_scope('dense1',reuse=reuse):
            weight = tf.get_variable('weight',[100,256],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[256],initializer=tf.constant_initializer(0.))
            
            dense1 = tf.nn.relu(tf.matmul(z,weight) + bias)
        
        with tf.variable_scope('dense2',reuse=reuse):
            weight = tf.get_variable('weight',[256,512],
                                     initializer=tf.truncated_normal_initializer(mean=0,stddev=0.02))
            bias = tf.get_variable('bias',[512],initializer=tf.constant_initializer(0.))
            
            dense2 = tf.nn.relu(tf.matmul(dense1, weight)+bias)
        
        with tf.variable_scope('dense3',reuse=reuse):
            weight = tf.get_variable('weight',[512,1024],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[1024],initializer=tf.constant_initializer(0.))
            
            dense3 = tf.nn.relu(tf.matmul(dense2, weight) + bias)
        
        with tf.variable_scope('output',reuse=reuse):
            weight = tf.get_variable('weight',[1024,784],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[784],initializer=tf.constant_initializer(0.))
            
            output = tf.nn.tanh(tf.matmul(dense3, weight) + bias)
        
        return output

In [ ]:
def discriminator(image, drop_out, reuse=False):
    with tf.variable_scope('discriminator',reuse=reuse):
        with tf.variable_scope('dense1',reuse=reuse):
            weight = tf.get_variable('weight',[784,1024],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[1024],initializer=tf.constant_initializer(0.))
            
            dense1 = tf.nn.relu(tf.matmul(image,weight) + bias)
            dense1 =tf.nn.dropout(dense1,drop_out)
        
        with tf.variable_scope('dense2',reuse=reuse):
            weight = tf.get_variable('weight',[1024,512],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[512],initializer=tf.constant_initializer(0.))
            
            dense2 = tf.nn.relu(tf.matmul(dense1,weight) + bias)
            dense2 =tf.nn.dropout(dense2,drop_out)
            
        with tf.variable_scope('dense3',reuse=reuse):
            weight = tf.get_variable('weight',[512,256],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[256],initializer=tf.constant_initializer(0.))
            
            dense3 = tf.nn.relu(tf.matmul(dense2,weight) + bias)
            dense3 =tf.nn.dropout(dense3,drop_out)
        
        with tf.variable_scope('output',reuse=reuse):
            weight = tf.get_variable('weight',[256,1],
                                     initializer=tf.truncated_normal_initializer(mean=0.,stddev=0.02))
            bias = tf.get_variable('bias',[1],initializer=tf.constant_initializer(0.))
            
            output = tf.nn.sigmoid(tf.matmul(dense3,weight) + bias)
        
        return output
           

In [ ]:
# save image
def save_f_image(index, z_sample):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(z_sample):  # [i,samples[i]] imax=16
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='gray')
    plt.savefig('{}.png'.format(str(index).zfill(3)), bbox_inches='tight')
    plt.close(fig)

In [ ]:
x = tf.placeholder(tf.float32, shape=(None,784))
z = tf.placeholder(tf.float32, shape=(None,100))
drop_out = tf.placeholder(tf.float32)

f_image = generator(z)
d_real = discriminator(x,drop_out)

d_fake = discriminator(f_image,drop_out,reuse=True)
'''
d_loss = -tf.reduce_mean(tf.log(d_real) + tf.log(1. - d_fake))
g_loss = -tf.reduce_mean(tf.log(d_fake))
'''
eps = 1e-2
d_loss = tf.reduce_mean(-tf.log(d_real + eps) - tf.log(1. - d_fake + eps))
g_loss = tf.reduce_mean(-tf.log(d_fake + eps))
                    

In [ ]:
batch_size = 100
learning_rate = 0.0002
num_epochs = 200

train_set = (mnist.train.images - 0.5)/0.5
test_z = np.random.normal(0,1,(16,100))

t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'discriminator' in var.name]
g_vars = [var for var in t_vars if 'generator' in var.name]
print(d_vars)
print(g_vars)
d_train = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()



epoch = 0
index =0
while epoch <= num_epochs:
    g_losses = []
    d_losses = []
    
    if epoch % 1 == 0:
        fake_image = sess.run(f_image,feed_dict={z:test_z,drop_out:0.})
        save_f_image(index, fake_image)
        index += 1
    for i in range(0,train_set.shape[0],batch_size):
        sess.graph.finalize()
        input_x = train_set[i:i+batch_size]
        input_z = np.random.normal(0,1,(input_x.shape[0],100))
        
        sess.run(d_train, feed_dict={x:input_x, z:input_z,drop_out:0.3})
        loss_d = sess.run(d_loss,feed_dict={x:input_x,z:input_z,drop_out:0.3})
        d_losses.append(loss_d)
        
        input_z = np.random.normal(0,1,(input_x.shape[0],100))
        
        sess.run(g_train,feed_dict={z:input_z,drop_out:0.3})
        loss_g = sess.run(g_loss,feed_dict={z:input_z,drop_out:0.3})
        g_losses.append(loss_g)
    print("@epoch of {}, D_loss: {}, G_loss: {}".format(epoch+1,np.mean(d_losses),np.mean(g_losses)))
    epoch += 1

save_path = saver.save(sess, './gan_model.ckpt')
